## Code for CO2 Bubbling -> ECMS Demonstration

Go to *Platformio > Devices* to find COM port address.

In [ ]:
%load_ext autoreload
%autoreload 2

from src.pump_controller import ECMSController

ecms_handler = ECMSController(COM="/dev/cu.usbmodem11201", sim=False)

## Set CO2 Bubbling & Dilution Parameters

In [ ]:
# Dilution parameters
total_volume = 30 #ml
khco3_volume = 0.1 * total_volume #ml
water_volume = total_volume - khco3_volume #ml

# CO2 parameters
CO2_bubbling = 0.5 #mins

# Tube lengths
chemical_to_mixture = 600 #mm
water_to_mixture = 600 #mm
bubbler_to_ecms = 1000 #mm

In [ ]:
ecms_handler.releaseCO2(duration=10)

## Prime Water & Chemical Lines

In [ ]:
# Only run if needed!
ecms_handler.addChemical(tube_length=chemical_to_mixture, overpump=1.5)
ecms_handler.addWater(fluid_vol=total_volume, tube_length=water_to_mixture, overpump=1.5)

ecms_handler.sendToWaste()

## Run Procedure

In [ ]:
# Create mixture
ecms_handler.addChemical(fluid_vol=khco3_volume)
ecms_handler.addWater(fluid_vol=water_volume)

# Bubble CO2
ecms_handler.releaseCO2(duration=CO2_bubbling)

# Transfer for measuring
ecms_handler.sendToWaste()
ecms_handler.sendToPH(tube_length=100)

# Take measurement
while not ecms_handler.checkPH():
    ecms_handler.releaseCO2(duration=CO2_bubbling/3)
    ecms_handler.sendToWaste()
    ecms_handler.sendToPH(tube_length=100)

# Remove electrolyte and wet pH probe
ecms_handler.sendToWaste()
ecms_handler.refreshWater()

# Transfer to ECMS
ecms_handler.transferToECMS(fluid_vol=total_volume-ecms_handler.pH_volume, tube_length=bubbler_to_ecms)

# Empty pH lines
ecms_handler.sendToPH()